In [1]:
!pip install PyTDC


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\pahad\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [3]:
!pip install tensorflow deepchem


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\pahad\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [4]:
from tdc import utils
names = utils.retrieve_benchmark_names('ADMET_Group')

In [5]:
import deepchem as dc
from tdc.single_pred import ADME
from tdc.single_pred import Tox

# Load the dataset
data = Tox(name = 'LD50_Zhu')
split = data.get_split()


# Featurize data
featurizer = dc.feat.OneHotFeaturizer()
train_features = featurizer.featurize(split['train']['Drug'])
test_features = featurizer.featurize(split['test']['Drug'])

# Inspect the shape of the features
print("Train features shape:", train_features.shape)
print("Test features shape:", test_features.shape)

Downloading...
100%|███████████████████████████████████████████████████████████████████████████████| 707k/707k [00:03<00:00, 188kiB/s]
Loading...
Done!


Train features shape: (5170,)
Test features shape: (1477,)


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert SMILES to one-hot encoded sequences
def one_hot_smiles(smiles_list, max_length):
    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts(smiles_list)
    sequences = tokenizer.texts_to_sequences(smiles_list)
    one_hot = pad_sequences(sequences, maxlen=max_length, padding='post')
    return one_hot, tokenizer.word_index

# Determine max length for padding
max_smiles_length = max(len(s) for s in split['train']['Drug'])

# Apply one-hot encoding
train_smiles = split['train']['Drug']
test_smiles = split['test']['Drug']
train_features, _ = one_hot_smiles(train_smiles, max_smiles_length)
test_features, _ = one_hot_smiles(test_smiles, max_smiles_length)

print("Train features shape:", train_features.shape)
print("Test features shape:", test_features.shape)


Train features shape: (5170, 174)
Test features shape: (1477, 174)


In [7]:
train_labels = split['train']['Y']
test_labels = split['test']['Y']

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_regressor.fit(train_features, train_labels)

# Predict on the test set
rf_predictions = rf_regressor.predict(test_features)

In [9]:
# Calculate MSE and RMSE
mse = mean_squared_error(test_labels, rf_predictions)
rmse = mse ** 0.5
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")


Mean Squared Error: 0.482347460685289
Root Mean Squared Error: 0.6945123905916215


In [12]:
from deepchem.feat import CircularFingerprint

# Initialize the CircularFingerprint featurizer
featurizer = CircularFingerprint(size=2048)  # You can adjust the size as needed

# Featurize the data
train_features = featurizer.featurize(split['train']['Drug'])
test_features = featurizer.featurize(split['test']['Drug'])

# Check the shape again
print("Train features shape:", train_features.shape)
print("Test features shape:", test_features.shape)

# If the shapes are now correct, proceed with model training
model.fit(train_features, split['train']['Y'])


Train features shape: (5170, 2048)
Test features shape: (1477, 2048)


RandomForestRegressor(random_state=42)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Extract labels for training and testing
train_labels = split['train']['Y']
test_labels = split['test']['Y']

# Train the model
rf_regressor.fit(train_features, train_labels)

# Predict on the test set
rf_predictions = rf_regressor.predict(test_features)

# Evaluate the model using Mean Squared Error (MSE) and Root Mean Squared Error (RMSE)
mse = mean_squared_error(test_labels, rf_predictions)
rmse = mse ** 0.5

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")


Mean Squared Error (MSE): 0.37210263630074736
Root Mean Squared Error (RMSE): 0.6100021608984245
